# Seaching for keyword tags

Exploring how to use a menu to query the keyword tags returned by the DataONE API. 
To perform the query, run each cell till you get to the bottom and then choose an LTER Core Area from the dropdown menu and hit the "Search" button.

ELD

7/13/2020

In [1]:
import requests
import ipywidgets as widgets

# xml is a standard python library 
import xml.etree.ElementTree as ET
# lxml extends it with some things that make parsing and printing nicer
import lxml.etree as etree

In [2]:
# Run the query and transform the response into XML that can be navigated
# Possible member node interactions are described in
# https://releases.dataone.org/online/api-documentation-v2.0.1/apis/MN_APIs.html#module-MNQuery
# Fields for the searches are described in 
# https://releases.dataone.org/online/api-documentation-v2.0.1/design/SearchMetadata.html?highlight=field#querying-the-solr-index

url_nga = ('https://cn.dataone.org/cn/v2/query/solr/?fl=title,keywords,id'
                 '&q=keywords:NGA%20LTER'
                 '&sort=id%20asc&rows=60&start=0')
resp = requests.get(url_nga)
resp.content

# Transform response to an element tree
parser = etree.XMLParser(remove_blank_text=True)
root = ET.fromstring(resp.content, parser)


In [3]:
# Define functions that will look for a keyword in the DataONE response. includes a little test.

keyword = 'marine'

def compare_words(word1, word2):
    """ indentify matches with a little flexibility regarding capitolization and substrings """
    match = False
    if word1.lower() in word2.lower() or word2.lower() in word1.lower():
        match = True
    return match

def print_matches(tag, root):
    """ print the titles of records that have matching keywords """
    matches = 0
    for doc in root.iter("doc"):
        keywords = []
        for child in doc:
            if "name" in child.keys() and child.attrib["name"] == "title":
                title = child.text
            if "name" in child.keys() and child.attrib["name"] == "keywords":
                for keyword in child:
                    keywords.append(keyword.text)
        if any(compare_words(tag,word) for word in keywords):
            matches +=1
            print(f'"{title}"')
            
    return matches

n = print_matches(keyword, root)
print(f'\n{n} matching records')

"Gulf of Alaska ROMS-COBALT Hindcast Simulation 1980 - 2013"
"Ocean currents measured by R/V Sikuliaq's Shipboard Acoustic Doppler Current Profiler (SADCP) during the Northern Gulf of Alaska LTER Spring 2018 cruise"
"Particle abundance and volume size distributions from an Underwater Vision Profiler 5 aboard seasonal cruises for the Northern Gulf of Alaska LTER site 2018"
"Marine bird survey observation and density data from Northern Gulf of Alaska LTER cruises, 2018"

4 matching records


In [4]:
# Use the widgets to interactively choose the LTER Core Area to ask about.
 
def on_button_clicked(b):
    with output:
        print(f'Searching for {menu.value}')
        n = print_matches(menu.value, root)
        print(f'\n{n} matching records\n\n')


def on_clear_clicked(b):
    with output:
        output.clear_output()

output = widgets.Output()
button = widgets.Button(description="Seach Now")
button.on_click(on_button_clicked)
clear_button = widgets.Button(description="Clear")
clear_button.on_click(on_clear_clicked)
menu = widgets.Dropdown(options=['Disturbance Pattern', 
                                 'Primary Production', 
                                 'Movement of Inorganic Matter',
                                 'Movement of Organic Matter',
                                 'Population Studies'],
                        value='Disturbance Pattern',
                        description='LTER Core Area:',
                        disabled=False)


In [5]:
# Run it
              
display(menu)
display(button, output)
display(clear_button)


Dropdown(description='LTER Core Area:', options=('Disturbance Pattern', 'Primary Production', 'Movement of Ino…

Button(description='Seach Now', style=ButtonStyle())

Output()

Button(description='Clear', style=ButtonStyle())